In [1]:
import sys
#sys.path.insert(1, '/home/linus/projects/fdl/crisp')

In [2]:
import os
import pandas as pd
while os.getcwd() != '/home/jupyter/crisp':
    os.chdir("..")
%pwd

'/home/jupyter/crisp'

In [3]:
from models.SimpleNonLinearInvariantRiskMinimization import SimpleNonLinearInvariantRiskMinimization
from dataio.DataFrameDataset import *
from dataio.datasets import *

In [4]:
from synthetic.facebook_synthetic_data_generator import generator_example

dim_inv = 2
dim_spu = 2
n_exp_train = 6 #int(2e2)
n_exp_test = 6
n_env = 3
save_dir = 'data/synthetic'

In [5]:
config = {
    "name": "Example Experiment for AH casual ensemble",
    "short_name": "ah_experiment_notebook",
    "bucket_project": "fdl-us-astronaut-health",
    "bucket_name": "ah_21_data",
    "bucket_path": "gs://ah_21_data",
    "verbose": 1,
    "test_val_split": [0.1, 0.1],
    "per_variant_experiment": False,
    "data_options": {
        #'dataset_fp': '../data/test_multiclass.pkl',
        'dataset_fp' : None,
        'output_data_regime' : ["real-valued", "binary", "binary", "real-valued", "multi-class"],
        'subject_keys': 'Subj_ID',
        'targets': ['Target'],
 #        'predictors': ['All'],
        'predictors': None,
        'environments': ['env_split'],
        'exclude': ['Subj_ID'],
        'synthetic_train_test_split' : True,
    },
    "feature_selection_options": {
        "max_features": 20,
        "verbose": 0,
        "seed": 12
    },
    "ensemble_options": {
        "models": ["ERM", "RF", "ICP", "IRM", "DCF", "ITE", "LIRM", "NLICP"]
    },
    "use_cloud":False,
    "results_directory": "results/"
}
data_config = config['data_options']

In [6]:
SNLIRM_options = config.get("ensemble_options").get('LIRM', {})
SNLIRM_args = {
            "use_icp_initialization": False,
            "verbose": 1,
            "n_iterations": 3000, # 1000
            "seed": 0,
            "lr": 0.005, # 0.001
            "cuda": False
        }
SNLIRM_args.update(SNLIRM_options)

In [7]:
def get_dataset_for_synthetic_experiment(n_example, dim_inv, dim_spu, n_exp_train, n_exp_test, n_env, save_dir):
    # training data
    generator_example(n_example, dim_inv, dim_spu, n_exp_train, n_env ,save_dir, False)
    outfile_train = os.path.join(save_dir,"data_fb_example_%s_dim_inv_%s_dim_spu_%s_n_exp_%s_n_env_%s_test_%s.pickle" %(n_example, dim_inv, dim_spu, n_exp_train, n_env, False))

    # test data
    generator_example(n_example, dim_inv, dim_spu, n_exp_test, n_env ,save_dir, True)
    outfile_test = os.path.join(save_dir,"data_fb_example_%s_dim_inv_%s_dim_spu_%s_n_exp_%s_n_env_%s_test_%s.pickle" %(n_example, dim_inv, dim_spu, n_exp_test, n_env, True))
    
    return outfile_train, outfile_test

In [8]:
# lirm.loss_per_iteration

In [9]:
results = {}

for n_example in [1]:#,2,3,4,5]:
    output_data_regime = config["data_options"]["output_data_regime"][n_example-1]
    print("\n\nn_example:", n_example, "output_data_regime:", output_data_regime, "\n\n")
    
    outfile_train, outfile_test = get_dataset_for_synthetic_experiment(n_example, dim_inv, dim_spu, n_exp_train, n_exp_test, n_env, save_dir)
    df_train = pd.read_pickle(outfile_train)
    df_test = pd.read_pickle(outfile_test)

    config["data_options"]["dataset_fp_train"] = outfile_train
    config["data_options"]["dataset_fp_test"] = outfile_test
    config["data_options"]["predictors"] = list(df_train.columns[0:dim_inv+dim_spu])
    environment_datasets, val_dataset, test_dataset = get_datasets_for_experiment(config)

    SNLIRM_args["target"] = config["data_options"]["targets"]
    SNLIRM_args["output_data_regime"] = output_data_regime
    SNLIRM_args["columns"] = test_dataset.predictor_columns

    lirm = SimpleNonLinearInvariantRiskMinimization(environment_datasets, val_dataset, test_dataset, SNLIRM_args)
    
    lirm_results_dict = lirm.results()
    results[n_example] = lirm_results_dict



n_example: 1 output_data_regime: real-valued 


Environments variables: {'E0': 0.1, 'E1': 1.5, 'E2': 2}
Generated Synthetic Data according to the Facebook setup Example: 1
     df with  4  columns
Environments variables: {'E0': 0.1, 'E1': 1.5, 'E2': 2}
Generated Synthetic Data according to the Facebook setup Example: 1
     df with  4  columns
Running a per sample experiment
Using synthetic dataset train/test split
Loaded  3  train environments
Env  0  has  2  samples
X shape  (2, 4)  y shape  (2, 1)
Env  1  has  2  samples
X shape  (2, 4)  y shape  (2, 1)
Env  2  has  2  samples
X shape  (2, 4)  y shape  (2, 1)
Loaded test set, X shape: (6, 4)  y shape:  (6, 1)
Start simple nonlinear-IRM training procedure
logging accuracy and loss
logging accuracy and loss
logging accuracy and loss
logging accuracy and loss
logging accuracy and loss
logging accuracy and loss
logging accuracy and loss
logging accuracy and loss
logging accuracy and loss
logging accuracy and loss
logging accuracy and 

In [10]:
lirm.acc_per_iteration

[-0.0036234020948742174,
 -0.0938305036631597,
 -0.2195319438785115,
 -0.37758749177150386,
 -0.5646517629540619,
 -0.7776145254980265,
 -1.013761742132104,
 -1.2708345545173616,
 -1.5469940331625733,
 -1.8407467267443813,
 -2.15086656272024,
 -2.476309159635228,
 -2.81613405859363,
 -3.169445997436357,
 -3.535378601097494]

In [11]:
lirm.loss_per_iteration

[tensor(64.4530, grad_fn=<DivBackward0>),
 tensor(62.8323, grad_fn=<DivBackward0>),
 tensor(61.1626, grad_fn=<DivBackward0>),
 tensor(59.4455, grad_fn=<DivBackward0>),
 tensor(57.6937, grad_fn=<DivBackward0>),
 tensor(55.9270, grad_fn=<DivBackward0>),
 tensor(54.1683, grad_fn=<DivBackward0>),
 tensor(52.4405, grad_fn=<DivBackward0>),
 tensor(50.7632, grad_fn=<DivBackward0>),
 tensor(49.1518, grad_fn=<DivBackward0>),
 tensor(47.6169, grad_fn=<DivBackward0>),
 tensor(46.1652, grad_fn=<DivBackward0>),
 tensor(44.8001, grad_fn=<DivBackward0>),
 tensor(43.5225, grad_fn=<DivBackward0>),
 tensor(42.3310, grad_fn=<DivBackward0>)]